In [287]:
import numpy as np
import torch
import warnings
warnings.filterwarnings("ignore")

In [288]:
def print_operands(operands):
    for i, operand in enumerate(operands):
        print('{} shape:{}'.format(i, operand.shape))
def einsum_label_to_index(label):
    if (label == '.'):
        return 52
    NUM_OF_LETTERS = ord('z') - ord('a') + 1;
    return (ord(label) - ord('A')) if(label.isupper()) else (NUM_OF_LETTERS + (ord(label) - ord('a')))

In [289]:
einsum_label_to_index('a')

26

In [290]:
def np_unsqueeze(np_array, axis):
    shape = np.shape(np_array)
    if (axis > len(shape)):
        raise(" np_unsqueeze error! ")
    new_shape = []
    for i in range(len(shape)):
        if (i != axis):
            new_shape.append(slice(0, shape[i]))
        else:
            new_shape.append(np.newaxis)
            new_shape.append(slice(0, shape[i]))
    if (axis == len(shape)):
        new_shape.append(np.newaxis)
    return np_array[new_shape]

In [291]:
a = np.random.randn(2,3)
np_unsqueeze(a, 2).shape

(2, 3, 1)

In [292]:
def np_bmm(array1, array2, _lro, _lo, _ro, _sum_dim_, out_shape):
    import torch
#     print(_lro, _lo, _ro, _sum_dim_, out_shape)
    #     out = np_bmm(array1, array2, [3],[0,1,2],[4],[5,6,7])
    #     array1 # (8, \5, 6, 7, \2, 3, 4, \1)  lro lo sum ro
    #     array2 # (8, \2, 3, 4, \10, \1, 1, 1) lro sum ro lo
    ###########################################################################
    split_dim = []
    for _idx in (_lro+_lo+_sum_dim_+_ro):
        if _idx in _lro:
            split_dim.append(0)
        if _idx in _lo:
            split_dim.append(1)
        if _idx in _sum_dim_:
            split_dim.append(2)
    
    lshape = [1]*3
    for i in range(len(split_dim)):
        lshape[split_dim[i]] *= np.shape(array1)[i]
    array1 = torch.tensor(np.reshape(array1, lshape))
    ###########################################################################
    split_dim = []
    for _idx in (_lro+_sum_dim_+_ro+_lo):
        if _idx in _lro:
            split_dim.append(0)
        if _idx in _sum_dim_:
            split_dim.append(1)
        if _idx in _ro:
            split_dim.append(2)
    
    rshape = [1]*3
    for i in range(len(split_dim)):
        rshape[split_dim[i]] *= np.shape(array2)[i]
    array2 = torch.tensor(np.reshape(array2, rshape))
#     print(lshape, rshape, array1.shape, array2.shape)
    ###########################################################################
    out = torch.bmm(array1, array2)
    out = out.numpy()
    out = np.reshape(out, out_shape)
    return out

In [293]:
# result, permuted_operands[i], sum_dims, false
def maybe_wrap_dim(dim, ndims):
    _min = -1*ndims
    _max = ndims - 1
#     print(dim, _min, _max)
    if (dim<_min or dim>_max):
        raise("seen error")
    if (dim < 0):
        dim += ndims
    return dim
def sumproduct_pair(left_, right_, sum_dims_, keep_dim_):
#     left_ = np.random.randn(5, 6, 7, 8, 1, 2, 3, 4)
#     right_ = np.random.randn(1, 1, 1, 8, 10, 2, 3, 4)
#     sum_dims_ = [5,6,7]
#     keep_dim_ = False
#     print('*'*80)
#     print(np.shape(left_))
#     print(np.shape(right_))
#     print(sum_dims_)

    dim = len(left_.shape)

    # dim_list_to_bitset(sum_dims_, dim)
    seen = [-1]*(max(sum_dims_)+1)
    for i in sum_dims_:
        seen[i] = (maybe_wrap_dim(i, dim))
#     print('\n')
#     print('sum_dims:', seen)
    sum_dims = seen

    lro = []
    lo = []
    ro = []
    lro_size = 1
    lo_size = 1
    ro_size = 1
    sum_size = 1
    left = left_
    right = right_

    for i in range(dim):
        sl = (left.shape[i] > 1)
        sr = (right.shape[i] > 1)
        if (sum_dims[i] != -1):
            if (sl and sr):
                sum_size *= left.shape[i]
            elif (sl):
                left = np.sum(left, i, keepdims=True)
            elif (sr):
                right = np.sum(right, i, keepdims=True)
        elif (sl and sr):
            lro.append(i)
            lro_size *= left.shape[i]
        elif (sl):
            lo.append(i)
            lo_size *= left.shape[i]
        else:
            ro.append(i)
            ro_size *= right.shape[i]
#     print('\n')
#     print('lro:', lro, 'lo:', lo, 'ro:', ro, 'lro_size:', lro_size, 'lo_size:', lo_size, 'ro_size:', ro_size, 'sum_size:', sum_size)

    out_size = []
    for i, _lro in enumerate(lro):
        out_size.append(left.shape[_lro])
    for i, _lo in enumerate(lo):
        out_size.append(left.shape[_lo])
    for i, _sum_dims_ in enumerate(sum_dims_):
        out_size.append(1)
    for i, _ro in enumerate(ro):
        out_size.append(right.shape[_ro])
#     print('out_size:', out_size)

    lpermutation = lro.copy()
    lpermutation += lo
    lpermutation += sum_dims_
    lpermutation += ro
#     print('\n')
#     print('lpermutation:', lpermutation)

    rpermutation = lro.copy()
    rpermutation += sum_dims_
    rpermutation += ro
    rpermutation += lo
#     print('\n')
#     print('rpermutation:', rpermutation)

    opermutation = [-1]*(len(lro)+len(lo)+len(sum_dims_)+len(ro))
    for i, _it in enumerate(lro+lo+sum_dims_+ro):
        opermutation[i] = _it
#     print('\n')
#     print('opermutation:', opermutation)

    ###############################################################
    out_shape = []
    for i in range(len(lo+ro+lro)):
        out_shape.append(max(np.shape(left)[i], np.shape(right)[i]))
#     print('out_shape:', out_shape)
    ###############################################################

    left = np.transpose(left, tuple(lpermutation))
    right= np.transpose(right, tuple(rpermutation))
#     print('\n')
#     print(left.shape, right.shape)

    _sum_dim = []
    for i in range(len(np.shape(left))):
        if i not in (lro+lo+ro):
            _sum_dim.append(i)
    bmm_out = np_bmm(left, right, lro, lo, ro, _sum_dim, out_shape)
    return bmm_out

In [294]:
left_ = np.random.randn(5, 6, 7, 8, 1, 2, 3, 4)
right_ = np.random.randn(1, 1, 1, 8, 10, 2, 3, 4)
sum_dims_ = [5,6,7]
keep_dim_ = False
out = sumproduct_pair(left_, right_, sum_dims_, keep_dim_)
print(out.shape)

(5, 6, 7, 8, 10)


In [302]:
def einsum(equation, operands):
    op_labels = [[]]
    lhs = equation.split('->')[0]
    curr_op = 0
    ELLIPSIS = 52
    found_ell = False
    for i, label in enumerate(lhs):
        if (label == ' '):
            continue
        elif (label == '.'):
            op_labels[curr_op].append(ELLIPSIS)
            found_ell = True
        elif (label == ','):
            curr_op += 1
            found_ell = False
            op_labels.append([])
        else:
            op_labels[curr_op].append(einsum_label_to_index(label))
    ###################################################print
#     print("op_labels：\n", op_labels)
#     print('\n')
    ###################################################print
    
    TOTAL_LABELS = 52
    label_count = [0]*TOTAL_LABELS
    ell_num_dim = 0
    for i, operand in enumerate(operands):
        labels = op_labels[i]
        shape = np.shape(operand)
        ndims = len(shape)
        nlabels = len(labels)
        has_ellipsis = False

        for label in labels:
            if (label == ELLIPSIS):
                nlabels -= 1
                has_ellipsis = True
                ell_num_dim = max(ell_num_dim, ndims - nlabels) # 维度数量 - 除了.的字母数量 = 最大的.的数量
            else:
                label_count[label] += 1
    ###################################################print
#     print('label_count:')
#     for i in range(TOTAL_LABELS):
#         if (label_count[i] != 0):
#             print("{}:{}  ".format(i, label_count[i]), end='')
#     print('\n')
    ###################################################print

    label_perm_index = [-1]*TOTAL_LABELS
    perm_index = 0
    ell_index = 0
    found_ell = False

    if (len(equation.split('->')) == 1):
        perm_index = ell_num_dim
        found_ell = True
        for label, _label_count in enumerate(label_count):
            if (_label_count == 1):
                label_perm_index[label] = perm_index # 默认输入里.的维度是输出的维度，把所有字母排在后面
                perm_index += 1
    else:
        rhs = equation.split('->')[1]
        for i, label in enumerate(rhs):
            if (label == ' '):
                continue
            elif (label == '.'):
                ell_index = perm_index
                perm_index += ell_num_dim # 如果输出也有点就在中间留出 ell_num_dim 个位置
                found_ell = True
            else:
                index = einsum_label_to_index(label)
                label_perm_index[index] = perm_index # 首先留出输出字母的位置
                perm_index += 1
    ###################################################print
#     print('label_perm_index:')
#     for i in range(TOTAL_LABELS):
#         if (label_perm_index[i] != -1):
#             print("{}:{}  ".format(i, label_perm_index[i]), end='')
#     print('\n')
    ###################################################print
    out_size = perm_index
    if (found_ell is False):
        ell_index = perm_index # ell_index 是存的输出字母的位置
        perm_index += ell_num_dim # 如果输出没有点就在最后留出 ell_num_dim 个位置， ell_num_dim 不一定不是0，取决于输入

    for label in range(TOTAL_LABELS):
        if (label_count[label]>0 and label_perm_index[label] == -1):
            label_perm_index[label] = perm_index # label_perm_index 不存在输出字母串的输入字母在最后这些位置
            perm_index += 1
    ###################################################print
#     print('label_perm_index:')
#     for i in range(TOTAL_LABELS):
#         if (label_perm_index[i] != -1):
#             print("{}:{}  ".format(i, label_perm_index[i]), end='')
#     print('\n')
    ###################################################print

    permuted_operands = []
    for i, operand in enumerate(operands):
        perm_shape = [-1]*perm_index
        label_dim = [-1]*TOTAL_LABELS
        labels = op_labels[i]
        original_sizes = np.shape(operand)

        j = 0
        for label in labels:
            operand = operands[i]
            if (label == ELLIPSIS):
                num_missing_dim = ell_num_dim - (len(original_sizes) - len(labels) + 1)
                for k in range(num_missing_dim):
                    operands[i] = np_unsqueeze(operand, j) # 这个能补充到 perm_index 的维度, perm_index 是输出字母 + 一个或多个输入最大.数 + 输入字母
                for k in range(ell_num_dim):
                    perm_shape[ell_index + k] = j # perm_shape （代表每一个输入）里是先排 输入.的位置
                    j += 1
            elif (label_dim[label] != -1):
                dim = label_dim[label]
                operand = np.diagonal(operand, offset=0, axis1=dim, axis2=j)
                operands[i] = np.moveaxis(operand, -1, dim)
            else:
                label_dim[label] = j
                perm_shape[label_perm_index[label]] = j
                j += 1
        # [0,1,2,3,4,5,6,7] -> [3,4,-1,-1,0,1,2,-1]
        ###################################################print
#         print('perm_shape:', perm_shape)
        ###################################################print
        for perm_shape_i, index in enumerate(perm_shape):
            operand = operands[i]
            if (index == -1):
                operands[i] = np_unsqueeze(operand, len(np.shape(operand)))
                perm_shape[perm_shape_i] = j
                j += 1
        # [3,4,-1,-1,0,1,2,-1] -> [3,4,5,6,0,1,2,7]
        ###################################################print
#         print('perm_shape:', perm_shape)
        ###################################################print
        operand = operands[i]
        operand = np.transpose(operand, tuple(perm_shape))
        permuted_operands.append(operand)
    ###################################################print
#     print_operands(permuted_operands)
    ###################################################print

    dim_last_op = [0]*perm_index
    has_zero_size_dim = False
    for dim in range(perm_index):
        broadcast_size = permuted_operands[0].shape[dim]
        for i in range(1, len(operands)):
            dim_size = permuted_operands[i].shape[dim]
            if (broadcast_size!= dim_size and broadcast_size!=1 and dim_size!=1):
                raise("near dim_last_op")
            if (dim_size != 1):
                broadcast_size = dim_size
                dim_last_op[dim] = i
        has_zero_size_dim = has_zero_size_dim or (broadcast_size == 0)
    ###################################################print
#     print('\n')
#     print('dim_last_op:', dim_last_op)
#     print('has_zero_size_dim:', has_zero_size_dim)
    ###################################################print
        
        
    result = permuted_operands[0]
    # has_zero_size_dim = True
    if (has_zero_size_dim):
        out_shape = [-1]*out_size
        for i in range(out_size):
            out_shape[i] = permuted_operands[dim_last_op[i]].shape[i]
        return np.zeros(out_shape)

    dim = out_size
    for i in range(dim, perm_index):
        if (dim_last_op[i] == 0):
            if (result.shape[dim] == 1):
                result = np.squeeze(result, dim)
                dim -= 1
            else:
                result = np.sum(result, dim)
                dim -= 1
        dim += 1
        
    for i in range(1, len(permuted_operands)):
        sum_dims = []
        dim = out_size
        for j in range(dim, perm_index):
            if (dim_last_op[j] < i):
                permuted_operands[i] = np.squeeze(permuted_operands[i], dim)
                dim -= 1
            elif (dim_last_op[j] == i):
                if (result.shape[dim] == 1):
                    permuted_operands[i] = np.sum(permuted_operands[i], dim)
                    result = np.squeeze(result, dim)
                    dim -= 1
                else:
                    sum_dims.append(dim)
            dim += 1
#         print('sum_dims:',sum_dims)
#         print('compute:', np.shape(result), np.shape(permuted_operands[i]))
        if (len(sum_dims) == 0):
#             print('直接算')
            result = np.multiply(result, permuted_operands[i])
        elif (len(sum_dims) == len(result.shape)):
#             print('flatten')
            result = result.flatten().dot(permuted_operands[i].flatten())
        else:
#             print('sumproduct_pair')
            result = sumproduct_pair(result, permuted_operands[i], sum_dims, False)
    return result

In [303]:
operands = []
a1 = np.random.randn(1,2,3,4,5,6)
a2 = np.random.randn(1,2,3,4,7,8,9)
a3 = np.random.randn(1,2,3,4,8,10)
operands.append(a1)
operands.append(a2)
operands.append(a3)
equation = "abcdef,abcdghi,abcdhk->efghki" # 5,6,7,8,10
out = einsum(equation, operands)
np_out = torch.einsum(equation, torch.tensor(a1), torch.tensor(a2), torch.tensor(a3))
print(out.shape, (np_out-out).sum())

(5, 6, 7, 8, 10, 9) tensor(1.1369e-12, dtype=torch.float64)


In [304]:
operands = []
a1 = np.random.randn(1,2,3,4,5,6)
a2 = np.random.randn(1,2,3,4,7,8,9)
operands.append(a1)
operands.append(a2)
equation = "abcdef,abcdghi->egh" # 5,6,7,8,10
out = einsum(equation, operands)
np_out = torch.einsum(equation, torch.tensor(a1), torch.tensor(a2))
print(out.shape, (np_out-out).sum())

(5, 7, 8) tensor(6.4393e-14, dtype=torch.float64)


In [307]:
operands = []
a1 = np.random.randn(1,2,3,4,5,6)
a2 = np.random.randn(1,2,3,4,7,8,9)
operands.append(a1)
operands.append(a2)
equation = "a.ef,a.ghi->efgh" # 5,6,7,8,10
out = einsum(equation, operands)
np_out = torch.einsum(equation, torch.tensor(a1), torch.tensor(a2))
print(out.shape, (np_out-out).sum())

RuntimeError: '.' must only occur in ellipsis, operand 0

In [306]:
operands = []
operands.append(np.random.randn(1,2,3,4,5,6))
operands.append(np.random.randn(1,2,3,4,7,8,9))
equation = "abcdef,abcdghi->.gh" # 5,6,7,8,10
out = einsum(equation, operands)
print(out.shape)

(7, 8)


In [300]:
operands = []
operands.append(np.random.randn(1,2,3,4,5,6))
operands.append(np.random.randn(1,2,3,4,7,8,9))
operands.append(np.random.randn(1,2,3,4,8,10))
equation = "a.def,a.dghi,a.dhk->eghk" # 5,6,7,8,10
out = einsum(equation, operands)
print(out.shape)

(5, 7, 8, 10)
